In [ ]:
%pip install -r requirements.txt

In [1]:
# import gymnasium as gym
import numpy as np
import pandas as pd

from stable_baselines3 import PPO
from StorageEnvEffLos_rate import IdealStorageShutdownEnv

from tools import denormalize

In [2]:
# Read your CSV file (adjust the path if necessary)
df = pd.read_csv('./data/DayAheadSpotPrices.csv')

# Check the number of rows in the DataFrame to ensure it matches 8760
print("Number of data points:", len(df))

# Create a date range for the year 2017 with hourly frequency
timestamps = pd.date_range(start='2017-01-01 00:00:00', periods=len(df), freq='H')

# Add the timestamp column to the DataFrame
df['timestamp'] = timestamps

# Optionally, set the timestamp column as the DataFrame index (if needed)
df.set_index('timestamp', inplace=True)

# Extract the 'price' column and reshape to a 2D array
prices = df['2017'].values.reshape(-1, 1)
prices = prices.flatten()
prices = prices[3000:5000]

Number of data points: 8760


In [ ]:
# Suppose you already have a prices array (NumPy) or None
env_1 = IdealStorageShutdownEnv(
    S_max=6,
    Delta_max=0.25,
    theta_min=0.5, 
    theta_max=0.2,
    zeta=0.02,
    cEl=prices,
    penalty_ramping_lambda=10, 
    penalty_demand_lambda=40, 
    penalty_storage_lambda=30
)
model = PPO(
    'MlpPolicy',
    env_1,
    verbose=1,
    gamma=0.995,
)
model.learn(total_timesteps=2e5)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2e+03     |
|    ep_rew_mean     | -2.25e+05 |
| time/              |           |
|    fps             | 7494      |
|    iterations      | 1         |
|    time_elapsed    | 0         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2e+03       |
|    ep_rew_mean          | -2.15e+05   |
| time/                   |             |
|    fps                  | 4714        |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012004439 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entro

In [ ]:
# model.save("./model/ppo_energy_storage")